In [22]:
import numpy as np
import cv2


# Count Yokoi Connectivity Number

## primitive function $h$

- on downsampled lena
- use 4-connectivity

> Ch6 ppt p.47-48, textbook p.272-276

We define the function h as stated in the lecture ppt, which is:

\begin{equation*}
h(b, c, d, e) = 
\begin{cases}
q & \text{if } b = c \land (d \ne b \lor e \ne b) \\
r & \text{if } b = c \land (d = b \land e = b) \\
s & \text{if } b \neq c \\
\end{cases}
\end{equation*}

Which means if we have:

\begin{equation*}
\begin{bmatrix}
e & d \\
b & c 
\end{bmatrix}
\end{equation*}

Then for example, we output `q` if it looks like:

\begin{equation*}
\begin{bmatrix}
0 & 1 \\
1 & 1 
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
1 & 1 
\end{bmatrix}
\begin{bmatrix}
0 & 0 \\
1 & 1 
\end{bmatrix}
\end{equation*}

we output `r` if it looks like:

\begin{equation*}
\begin{bmatrix}
1 & 1 \\
1 & 1 
\end{bmatrix}
\end{equation*}

we output `s` if it looks like:

\begin{equation*}
\begin{bmatrix}
0 & 1 \\
1 & 0 
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 1 
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
1 & 0 
\end{bmatrix}
\end{equation*}



In [23]:
def h(b, c, d, e):
    if b == c and (d != b or e != b):
        return 'q'
    elif b == c and (d == b and e == b):
        return 'r'
    else:
        return 's'


## primitive function $f$

$f$ counts the number of arguments having a paricular value.

The symbol (label) $5$ stands for interior pixels, so we output $5$ if all the arguments are $r$.

For other cases, the connectivity number is <ins>the number of times a 4-connected neighbor has the same value but the corresponding 3-pixel corner neighborhood does not</ins>, if we represent this in a formula, it would be:

\begin{equation*}
f(a_1, a_2, a_3, a_4) = \#\{ a_k | a_k = q \}
\end{equation*}


In [24]:
def f(a_1, a_2, a_3, a_4):
    if a_1 == a_2 == a_3 == a_4 == 'r':
        return 5
    else:
        connectivity_number = 0
        for a_i in [a_1, a_2, a_3, a_4]: 
            if a_i == 'q':
                connectivity_number += 1
        return connectivity_number


In [25]:
img = cv2.imread('lena.bmp', 0)

[ WARN:0@43.773] global loadsave.cpp:248 findDecoder imread_('lena.bmp'): can't open/read file: check file path/integrity


## Downsampling lena from 512x512 to 64x64

1. binarize lena image as HW2
2. use 8x8 blocks as a unit
3. take top-most left pixel as downsampled data

Note: Result is a 64x64 matrix

### Step 1: Binarize the image

In [26]:
binarized_img = np.zeros(img.shape, np.int8)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        if img[i][j] >= 128:
            binarized_img[i][j] = 1

AttributeError: 'NoneType' object has no attribute 'shape'

### Step 2, 3: Downsample the image

First we initialize `downsampled_img` as a 64x64 matrix, with all elements being 0.

Then we iterate through each 8x8 block in the binarized image, and take the top-most left pixel as the downsampled data.

In [30]:
downsampled_img  = np.zeros((64, 64), np.int8)
for i in range(0, 64):
    for j in range(0, 64):
        downsampled_img[i][j] = binarized_img[i*8][j*8]

## Calculate Yokoi Connectivity Number

To calculate the Yokoi Connectivity number, we first iterate through each pixel in the downsampled image.

Then for each foreground pixel, we look at its 8 neighbors.

Here we have to define different cases, since for pixels on the boundary, some of their neighbors do not exist.
We classify the situation into 9 cases:

- top

    1. top-left
    2. top-right
    3. top
- bottom

    4. bottom-left
    5. bottom-right
    6. bottom
7. left
8. right
9. center

The indexing for pixels in a 3x3 neighborhood is:

\begin{equation*}
\begin{bmatrix}
x_7 & x_2 & x_6\\
x_3 & x_0 & x_1 \\
x_8 & x_4 & x_5
\end{bmatrix}
\end{equation*}

After we got the value of each pixel in the neighborhood, we can calculate $a_1, a_2, a_3, a_4$ by the primitive function $h$, these values are calculated by definition as follows:

\begin{equation*}
\begin{split}
a_1 = h(x_0, x_1, x_6, x_2) \\
a_2 = h(x_0, x_2, x_7, x_3) \\
a_3 = h(x_0, x_3, x_8, x_4) \\
a_4 = h(x_0, x_4, x_5, x_1) \\
\end{split}
\end{equation*}

Finally, we can calculate the connectivity number $y$ by the primitive function $f$, which is defined as:

\begin{equation*}
y = f(a_1, a_2, a_3, a_4)
\end{equation*}

After all the calculations, we print out the connectivity number $y$ for each foreground pixel, and print a blank space if the pixel is a background pixel. 

The printing part is done by concatenating the connectivity number $y$ for each pixel in a line, and print the line after each row is iterated.


In [27]:
for i in range(downsampled_img.shape[0]):
    line = ''
    for j in range(downsampled_img.shape[1]):
        if downsampled_img[i][j] == 1:
            # top
            if i == 0:
                # case 1: top-left
                if j == 0:  
                    x_7, x_2, x_6 = 0, 0, 0
                    x_3, x_0, x_1 = 0, downsampled_img[i][j], downsampled_img[i][j+1]
                    x_8, x_4, x_5 = 0, downsampled_img[i+1][j], downsampled_img[i+1][j+1]

                # case 2: top-right
                elif j == downsampled_img.shape[1] - 1:
                    x_7, x_2, x_6 = 0, 0, 0
                    x_3, x_0, x_1 = downsampled_img[i][j-1], downsampled_img[i][j], 0
                    x_8, x_4, x_5 = downsampled_img[i+1][j-1], downsampled_img[i+1][j], 0
                
                # case 3: top
                else:
                    x_7, x_2, x_6 = 0, 0, 0
                    x_3, x_0, x_1 = downsampled_img[i][j-1], downsampled_img[i][j], downsampled_img[i][j+1]
                    x_8, x_4, x_5 = downsampled_img[i+1][j-1], downsampled_img[i+1][j], downsampled_img[i+1][j+1]

            # bottom    
            if i == downsampled_img.shape[0] - 1:

                # case 4: bottom-left
                if j == 0:
                    x_7, x_2, x_6 = 0, downsampled_img[i-1][j], downsampled_img[i-1][j+1]
                    x_3, x_0, x_1 = 0, downsampled_img[i][j], downsampled_img[i][j+1]
                    x_8, x_4, x_5 = 0, 0, 0

                # case 5: bottom-right
                elif j == downsampled_img.shape[1] - 1:
                    x_7, x_2, x_6 = downsampled_img[i-1][j-1], downsampled_img[i-1][j], 0
                    x_3, x_0, x_1 = downsampled_img[i][j-1], downsampled_img[i][j], 0
                    x_8, x_4, x_5 = 0, 0, 0

                # case 6: bottom
                else:
                    x_7, x_2, x_6 = downsampled_img[i-1][j-1], downsampled_img[i-1][j], downsampled_img[i-1][j+1]
                    x_3, x_0, x_1 = downsampled_img[i][j-1], downsampled_img[i][j], downsampled_img[i][j+1]
                    x_8, x_4, x_5 = 0, 0, 0
            
            # pixels that does not belongs to top or bottom
            else:
                # case 7: left
                if j == 0:
                    x_7, x_2, x_6 = 0, downsampled_img[i-1][j], downsampled_img[i-1][j+1]
                    x_3, x_0, x_1 = 0, downsampled_img[i][j], downsampled_img[i][j+1]
                    x_8, x_4, x_5 = 0, downsampled_img[i+1][j], downsampled_img[i+1][j+1]

                # case 8: right
                elif j == downsampled_img.shape[1] - 1:
                    x_7, x_2, x_6 = downsampled_img[i-1][j-1], downsampled_img[i-1][j], 0
                    x_3, x_0, x_1 = downsampled_img[i][j-1], downsampled_img[i][j], 0
                    x_8, x_4, x_5 = downsampled_img[i+1][j-1], downsampled_img[i+1][j], 0

                # case 9: center
                else:
                    x_7, x_2, x_6 = downsampled_img[i-1][j-1], downsampled_img[i-1][j], downsampled_img[i-1][j+1]
                    x_3, x_0, x_1 = downsampled_img[i][j-1], downsampled_img[i][j], downsampled_img[i][j+1]
                    x_8, x_4, x_5 = downsampled_img[i+1][j-1], downsampled_img[i+1][j], downsampled_img[i+1][j+1]

            a_1 = h(x_0, x_1, x_6, x_2)
            a_2 = h(x_0, x_2, x_7, x_3)
            a_3 = h(x_0, x_3, x_8, x_4)
            a_4 = h(x_0, x_4, x_5, x_1)
            connectivity_number = f(a_1, a_2, a_3, a_4)
            line += str(connectivity_number)
        else:
            line += ' '
    print(line)

NameError: name 'downsampled_img' is not defined